# N8N Streaming

## Config

1. Node – When chat message received
- Make Chat Publicly Available: *`Yes`*
- Add Field > Response Mode: *`Streaming`*
- Copy Chat URL

2. Node – Respond to Webhook (return execution.id)
- Respond with: *`JSON`*
- Response Body (*Expression*):
  ```json
  {
    "executionId": "{{$execution.id}}"
  }
  ```
- Add Option > Response Mode: *`Streaming`*
- Rename: Respond ExecutionID

## Fetch tracing

1. use `Respond to webhook` and send back the `executionId`
2. use `sessionId` in chat?sessionId={UUID} and find first sessionId in GET /executions

In [ ]:
import requests
import uuid

In [ ]:
CHAT_ENDPOINT = "http://localhost:5678/webhook/7fb04e65-fe13-4de6-b5df-6239ce26c48c/chat"
def generate_webhook_url():
    unique_id = str(uuid.uuid4())
    return f"{CHAT_ENDPOINT}?sessionId={unique_id}"

In [ ]:
response = requests.post(
    generate_webhook_url(),
    json={"chatInput": "Hello from Jupyter Notebook"}
)
print(f"Status Code: {response.status_code}")
print(f"Response: {response.text}")

Status Code: 200
Response: {"type":"begin","metadata":{"nodeId":"cb75c5c5-83cb-42e3-8fa9-0f956091347a","nodeName":"Respond to Webhook","itemIndex":0,"runIndex":0,"timestamp":1767168909523}}
{"type":"item","content":"{\"executionId\":\"18\"}","metadata":{"nodeId":"cb75c5c5-83cb-42e3-8fa9-0f956091347a","nodeName":"Respond to Webhook","itemIndex":0,"runIndex":0,"timestamp":1767168909523}}
{"type":"end","metadata":{"nodeId":"cb75c5c5-83cb-42e3-8fa9-0f956091347a","nodeName":"Respond to Webhook","itemIndex":0,"runIndex":0,"timestamp":1767168909523}}
{"type":"begin","metadata":{"nodeId":"6b498cc8-4139-49c7-a9bc-7775ae27696b","nodeName":"AI Agent","itemIndex":0,"runIndex":0,"timestamp":1767168909528}}
{"type":"item","content":"Hello","metadata":{"nodeId":"6b498cc8-4139-49c7-a9bc-7775ae27696b","nodeName":"AI Agent","itemIndex":0,"runIndex":0,"timestamp":1767168912976}}
{"type":"item","content":"!","metadata":{"nodeId":"6b498cc8-4139-49c7-a9bc-7775ae27696b","nodeName":"AI Agent","itemIndex":0,"r

In [ ]:
import requests
import json

# AI Agent -> Enable Streaming
# Chat Note -> Response Mode: Streaming

with requests.post(
    generate_webhook_url(),
    json={"chatInput": "Give me a short recipe for pancakes."},
    stream=True,
    headers={"Accept": "text/event-stream"}  # schadet i.d.R. nicht; falls n8n SSE nutzt, ist das passend
) as r:
    r.raise_for_status()

    meta = []
    for line in r.iter_lines(decode_unicode=True):
        if not line:
            continue

        data_json = json.loads(line)
        if "content" in data_json:
            print(data_json["content"], end="", flush=True)
        else:
            meta.append(data_json)

print("\n\nMeta Information:")
print(meta)

{"executionId":"20"}Quick pancakes (serves 2–3, ~8–10 pancakes)

Ingredients
- 1 cup all-purpose flour
- 1 tbsp sugar
- 1 tsp baking powder
- 1/4 tsp salt
- 1 cup milk
- 1 large egg
- 2 tbsp melted butter or oil
- 1/2 tsp vanilla (optional)

Method
1. Whisk dry ingredients in a bowl. In another bowl, beat egg with milk, butter, and vanilla.
2. Pour wet into dry and stir until just combined (a few lumps are OK).
3. Heat a lightly oiled skillet over medium heat. Pour 1/4 cup batter per pancake.
4. Cook until bubbles form and edges set (~2 min), flip and cook 1–2 min more until golden.
5. Serve warm with syrup, fruit, or butter.

Quick tip: Don’t overmix the batter for fluffier pancakes.

Meta Information:
[{'type': 'begin', 'metadata': {'nodeId': 'cb75c5c5-83cb-42e3-8fa9-0f956091347a', 'nodeName': 'Respond to Webhook', 'itemIndex': 0, 'runIndex': 0, 'timestamp': 1767168966887}}, {'type': 'end', 'metadata': {'nodeId': 'cb75c5c5-83cb-42e3-8fa9-0f956091347a', 'nodeName': 'Respond to Webhook

In [ ]:
import requests
import json


API_KEY = ""
EXECUTION_ID = "119"

URL = f"http://localhost:5678/api/v1/executions/{EXECUTION_ID}?includeData=true"

headers = {
    "X-N8N-API-KEY": API_KEY
}

response = requests.get(URL, headers=headers)
response.raise_for_status()

with open(f"execution_{EXECUTION_ID}.json", "w") as f:
    json.dump(response.json(), f, indent=4)